#### Script to plot firing rate between dot offset and reward delivery for each neuron in a session.

**Prerequisites**
- Install python environment 'spikeinterface_fnt'
- Run script to create a data frame with harp timestamps aligned to key behavioural events.
- Run spike sorting pipeline for the session of interest.

**Inputs:**
- Output from Kilosort spike-sorting in a subfolder (specified by the user) within the session folder.
- Raw Open-ephys data within session folder.

**Key outputs**


**Overview** 
1. Read (or create) sorting analyzer object
2. Extract firing rates for all (good) units based on spike sorting


**Configure directory parameters**

In [26]:
import os
import glob
import spikeinterface.core as sc
import pandas as pd
import numpy as np
#==============================================================================

animal_ID = 'FNT099'
session_ID = '2024-05-13T11-03-59'

# path behavioural data on ceph repo
input_root_dir = "W:/projects/FlexiVexi/behavioural_data" 
output_root_dir = (
    "C:/Users/megan/Documents/sjlab/flexible-navigation-task/Data Analysis"
)

kilosort_subfolder = 'Kilosort3'
#==============================================================================

session_folder = os.path.join(input_root_dir, animal_ID, session_ID)
session_folder_intermediate_variables = os.path.join(output_root_dir, 'intermediate_variables', animal_ID, session_ID)

**Create sorting analyzer for session**

In [9]:
import spikeinterface.extractors as se

# Path to Kilosort output files within session folder
kilosort_folder = os.path.join(session_folder, kilosort_subfolder, 'sorter_output')

# Get output from spike sorting using Kilosort, keeping only good units (according to Kilosort's criteria)
sorting_KS = se.read_kilosort(folder_path=kilosort_folder,keep_good_only=True)
print(sorting_KS)

# Get path to Open-Ephys Record Node within session folder
matching_files = glob.glob(os.path.join(session_folder, '**', 'settings.xml'), recursive=True)
if matching_files:
    # Get the first matching file
    first_matching_file = matching_files[0]

    # Get the directory of the first matching file
    path_to_recording = os.path.dirname(first_matching_file)
else:
    print("No 'settings.xml' file found in the specified path.")
        
    # Get recording from open ephys
recording = se.read_openephys(folder_path=path_to_recording, stream_name = 'Record Node 102#Neuropix-PXI-100.ProbeA')

# Create sorting analyzer
sorting_analyzer = sc.create_sorting_analyzer(sorting_KS, recording)

KiloSortSortingExtractor: 110 units - 1 segments - 30.0kHz


estimate_sparsity:   0%|          | 0/3271 [00:00<?, ?it/s]

**Get firing rates of an example neuron**

In [27]:
import spikeinterface.qualitymetrics as sqm

# Example parameters
window_size = 0.05  # 50ms window
step_size = 0.025    # 25ms step (50% overlap)

def calculate_firing_rate_over_time(spike_times, window_size, step_size, total_time):
    """
    Calculate the firing rate over time using a sliding window.
    
    Parameters:
    - spike_times: array-like, spike times of the unit
    - window_size: float, size of the sliding window in seconds
    - step_size: float, step size for the sliding window in seconds
    - total_time: float, total duration of the recording in seconds
    
    Returns:
    - firing_rates: list, firing rates over time
    """
    firing_rates = []
    for start_time in np.arange(0, total_time, step_size):
        end_time = start_time + window_size
        spikes_in_window = np.sum((spike_times >= start_time) & (spike_times < end_time))
        firing_rate = spikes_in_window / window_size
        firing_rates.append(firing_rate)
    return firing_rates

# Specify the unit ID you are interested in
unit_ids = sorting_KS.get_unit_ids()

##firing_rate is a dict containing the unit IDs as keys,
## and their firing rates across segments as values (in Hz).
#firing_rate = sqm.compute_firing_rates(sorting_analyzer)

# Exclude neurons with firing rates less than 0.5Hz

# Get the spike times for the specified unit
spike_times = sorting_KS.get_unit_spike_train(unit_id)
total_time = max(spike_times) # total duration of the recording in seconds

# Calculate firing rate over time
firing_rate = calculate_firing_rate_over_time(spike_times, window_size, step_size, total_time)

**Plot changes in firing rate over time**